In [1]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
os.chdir('../')
from model import *
os.chdir('../../data/')

In [2]:
transactions = pd.read_pickle('compressed_data/transactions_train.pkl')
customers = pd.read_pickle('compressed_data/customers.pkl')
articles = pd.read_pickle('compressed_data/articles.pkl')

transactions['week'] = 104 - (transactions.t_dat.max() - transactions.t_dat).dt.days // 7

print('First week num: ', transactions.week.min(), '\nLast week num: ', transactions.week.max(), '\n')

First week num:  0 
Last week num:  104 



In [3]:
# Test week is week after last week in train data
test_week = transactions.week.max() + 1

# Filter transactions to last 10 weeks (most recent data)
transactions = transactions[transactions.week > transactions.week.max() - 10]

In [4]:
transactions.columns

Index(['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id',
       'week'],
      dtype='object')

In [5]:
# Load radek's candidates
candidates_last_purchase = pd.read_csv('candidates/radek_last_purchase.csv')
candidates_bestsellers = pd.read_csv('candidates/radek_bestsellers.csv')
bestsellers_previous_week = pd.read_csv('candidates/radek_bestsellers_previous_week.csv')

# Load my candidates

In [6]:
seasonal_candidates_loaded = pd.read_csv('candidates_200_ranks/seasonal_candidates_2019.csv')

## Submission Loop -- check best k for kaggle

In [7]:
columns_to_use = [
    'article_id', 
    'product_type_no', 
    'graphical_appearance_no', 
    'colour_group_code', 
    'perceived_colour_value_id',
    'perceived_colour_master_id', 
    'department_no', 
    'index_code',
    'index_group_no', 
    'section_no', 
    'garment_group_no', 
    'FN', 
    'Active',
    'club_member_status', 
    'fashion_news_frequency', 
    'age', 
    'postal_code', 
    'bestseller_rank'
]

model_params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'boosting_type': 'dart',
    'n_estimators': 1,
    'importance_type': 'gain'
}

In [8]:
# Get bestsellers from previous week
bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week['week'] == bestsellers_previous_week['week'].max()]['article_id'].tolist()

for k in range(20, 201, 10):
    print('k = ', k)

    seasonal_candidates = seasonal_candidates_loaded[seasonal_candidates_loaded.article_id.isin(transactions.article_id.unique())].head(k)
    
    seasonal_bestsellers = transactions[['customer_id', 'week']].drop_duplicates()
    seasonal_bestsellers['key'] = 1
    seasonal_candidates['key'] = 1

    seasonal_candidates = seasonal_bestsellers.merge(seasonal_candidates, on='key').drop(columns='key')
    seasonal_candidates['t_dat'] = '2020-07-15'
    seasonal_candidates['price'] = 0
    seasonal_candidates['sales_channel_id'] = 2
    seasonal_candidates.drop(columns=['seasonal_bestseller_rank', 'year'], inplace=True)

    # Prepare data for model
    train_X, train_y, test_X, test, train_baskets = prepare_data(
        transactions,
        bestsellers_previous_week,
        candidates=[candidates_last_purchase, candidates_bestsellers, seasonal_candidates], 
        features=[customers, articles], 
        cols_to_use=columns_to_use
        )
    
    # Train model
    ranker = train_model(
        train_X, 
        train_y, 
        train_baskets, 
        model_params, 
        columns_to_use, 
        show_importance=10
    )

    # Make submission
    make_submission(customers, test, test_X, ranker, bestsellers_last_week, f'submission_seasonal_{k}')

    del seasonal_candidates, seasonal_bestsellers, train_X, train_y, test_X, test, train_baskets, ranker



k =  20
Percentage of real transactions:  0.0779173253115634
Mergining features...
Done.
Sorting data...
Done.
Preparing for training...
Done.
Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.289176 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1104
[LightGBM] [Info] Number of data points in the train set: 23463051, number of used features: 18
Feature importance:
bestseller_rank 0.4100716459066416
article_id 0.30253876536813384
perceived_colour_value_id 0.061037614365297685
index_code 0.05987238543987213
colour_group_code 0.047949815441443465
product_type_no 0.04336769386964039
graphical_appearance_no 0.028464341074866513
index_group_no 0.018193087851052798
section_no 0.01288933290528505
department_no 0.011907930392236709
Starting submission process...
Calculating predictions...
Done.
Creating submission...
Done.
Savi

100%|██████████| 58.0M/58.0M [00:26<00:00, 2.27MB/s]


Successfully submitted to H&M Personalized Fashion RecommendationsSubmission saved and submitted to Kaggle.
k =  30
Percentage of real transactions:  0.0632228639718578
Mergining features...
Done.
Sorting data...
Done.
Preparing for training...
Done.
Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.706566 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1093
[LightGBM] [Info] Number of data points in the train set: 30100035, number of used features: 18
Feature importance:
article_id 0.33450062257272467
bestseller_rank 0.33304677346647465
index_code 0.07817529146444506
perceived_colour_value_id 0.07391767697007509
garment_group_no 0.05596199092297255
department_no 0.04431911734422342
colour_group_code 0.03364512598353286
product_type_no 0.030855932298192487
perceived_colour_master_id 0.010678867771919858
section_no 0.0024

100%|██████████| 58.0M/58.0M [00:26<00:00, 2.28MB/s]


Successfully submitted to H&M Personalized Fashion RecommendationsSubmission saved and submitted to Kaggle.
k =  40
Percentage of real transactions:  0.05311954981441688
Mergining features...
Done.
Sorting data...
Done.
Preparing for training...
Done.
Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.916320 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1086
[LightGBM] [Info] Number of data points in the train set: 36800401, number of used features: 18
Feature importance:
article_id 0.4554993780344747
bestseller_rank 0.2678667310428709
product_type_no 0.09473483281125142
index_code 0.07042183105463544
perceived_colour_value_id 0.05472643298540227
garment_group_no 0.026216124031750852
perceived_colour_master_id 0.017002602811175688
department_no 0.008002071431279475
graphical_appearance_no 0.005529995797159292
age 0.0
St

...

Successfully submitted to H&M Personalized Fashion RecommendationsSubmission saved and submitted to Kaggle.
k =  190
Percentage of real transactions:  0.01835598524582854
Mergining features...
Done.
Sorting data...
Done.
Preparing for training...
Done.
Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 2.426483 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1027
[LightGBM] [Info] Number of data points in the train set: 116243133, number of used features: 18
Feature importance:
article_id 0.6463723847757629
index_code 0.10172629611093659
product_type_no 0.07441617363989601
bestseller_rank 0.062096453918375746
perceived_colour_value_id 0.05132550341406754
department_no 0.019250585838487197
garment_group_no 0.01273509236338469
colour_group_code 0.012601143654221793
graphical_appearance_no 0.00986652062359689
section_no 0.0096

100%|██████████| 57.3M/57.3M [00:26<00:00, 2.25MB/s]


Successfully submitted to H&M Personalized Fashion RecommendationsSubmission saved and submitted to Kaggle.
k =  200
Percentage of real transactions:  0.01835598524582854
Mergining features...
Done.
Sorting data...
Done.
Preparing for training...
Done.
Training model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 2.323619 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1027
[LightGBM] [Info] Number of data points in the train set: 116243133, number of used features: 18
Feature importance:
article_id 0.6463723847757629
index_code 0.10172629611093659
product_type_no 0.07441617363989601
bestseller_rank 0.062096453918375746
perceived_colour_value_id 0.05132550341406754
department_no 0.019250585838487197
garment_group_no 0.01273509236338469
colour_group_code 0.012601143654221793
graphical_appearance_no 0.00986652062359689
section_no 0.0096

100%|██████████| 57.3M/57.3M [00:26<00:00, 2.24MB/s]


Successfully submitted to H&M Personalized Fashion RecommendationsSubmission saved and submitted to Kaggle.


In [9]:
!kaggle competitions submissions -c h-and-m-personalized-fashion-recommendations | head -n 21

fileName                                                     date                 description                                           status    publicScore  privateScore  
-----------------------------------------------------------  -------------------  ----------------------------------------------------  --------  -----------  ------------  
submission_seasonal_200.csv.gz                               2023-12-13 00:17:30  submission_seasonal_200                               pending                              
submission_seasonal_190.csv.gz                               2023-12-13 00:12:05  submission_seasonal_190                               complete  0.01815      0.01819       
submission_seasonal_180.csv.gz                               2023-12-13 00:06:18  submission_seasonal_180                               complete  0.01815      0.01819       
submission_seasonal_170.csv.gz                               2023-12-12 23:58:35  submission_seasonal_170                         